## Лабораторная работа №3 ONNX Runtime

### Сегментатор растений

In [121]:
import os
from keras_preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

image_folder = 'data/images'
mask_folder = 'data/masks'

image_filenames = os.listdir(image_folder)
image_filenames.sort()
mask_filenames = os.listdir(mask_folder)
mask_filenames.sort()

IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16
EPOCHS = 10

def load_images_and_masks(image_dir, mask_dir, img_height, img_width):
    images = []
    masks = []

    for image_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, image_name)
        img = load_img(img_path, target_size=(img_height, img_width))
        img = img_to_array(img) / 255.0
        images.append(img)

        mask_path = os.path.join(mask_dir, image_name)
        mask = load_img(mask_path, target_size=(img_height, img_width), color_mode="grayscale")
        mask = img_to_array(mask) / 255.0
        masks.append(mask)

    return np.array(images), np.array(masks)

In [122]:
images, masks = load_images_and_masks(image_folder, mask_folder, IMG_HEIGHT, IMG_WIDTH)

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

In [123]:
import random
import numpy as np

idx = random.randint(0, len(X_val) - 1)
input_image = X_val[idx]
input_image_expanded = np.expand_dims(input_image, axis=0)

In [124]:
import onnxruntime as ort
import numpy as np
import time

session = ort.InferenceSession("model.onnx")

input_name = session.get_inputs()[0].name

start_time = time.time()
output = session.run(None, {input_name: input_image_expanded})
end_time = time.time()

print(f"Время инференса: {end_time - start_time} секунд")

Время инференса: 0.18994784355163574 секунд


2024-12-16 15:24:15.918085 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
2024-12-16 15:24:15.918099 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute


In [125]:
import onnx
from onnxruntime import InferenceSession
from onnxruntime_tools import optimizer

model_path = "model.onnx"
model = onnx.load(model_path)

optimized_model = optimizer.optimize_model(model_path)

onnx.save_model(optimized_model.model, "optimized_model.onnx")

session = InferenceSession("optimized_model.onnx")

start_time = time.time()
output = session.run(None, {input_name: input_image_expanded})
end_time = time.time()

print(f"Время инференса: {end_time - start_time} секунд")


2024-12-16 15:24:16.863682 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
2024-12-16 15:24:16.863696 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
Model producer not matched: Expect pytorch,  Got tf2onnx 1.16.1 15c810. Please specify correct --model_type parameter.
2024-12-16 15:24:16.888518 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
2024-12-16 15:24:16.888541 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute


Время инференса: 0.1924302577972412 секунд
